In [ ]:
options(warn=-1)

library(tidyverse)
library(mygene)
library(feather)

refFlatFileUrl <- "https://gist.github.com/kdaily/2ed85e0dd3048fea8424b40243ddfa1c/raw/420086bd941962df66992667972c13462e504cc6/gencode.v24.primary_assembly.refFlat.txt"

In [ ]:
geneTable <-  readr::read_tsv(refFlatFileUrl, col_names=FALSE) %>%
              dplyr::select(ensembl_gene_id=X1) %>%
              dplyr::mutate(ensembl_gene_id=stringr::str_remove(ensembl_gene_id, "\\..*")) %>%
              dplyr::distinct()

print(geneTable)

In [ ]:
geneInfoRes <- mygene::getGenes(geneTable$ensembl_gene_id, fields=c("symbol", "name", "summary", "type_of_gene"))

In [ ]:
geneInfo <- geneInfoRes %>%
            as.data.frame() %>%
            as_tibble() %>%
            group_by(query) %>%
            ungroup()

# the group/ungroup don't do anything to the dataset, but since that's how it was originally, I kept it for the sake of fidelity

In [ ]:
geneTableMerged <- left_join(geneTable, 
                             geneInfo,
                             by=c("ensembl_gene_id"="query")) %>% as.data.frame()

In [ ]:
path <- "../output/gene_table_merged_R.feather"
write_feather(geneTableMerged, path)